# 1. Import Necessary Libraries

Make sure  Microsoft Visual C++ is installed on your pc

Extracting text from pdf and converting to csv

In [ ]:
"""
# PDF Text Extraction and Topic Modeling

This program extracts text from PDFs, cleans it, and performs topic modeling using BERTopic.

## Dependencies
- pdfplumber
- pandas
- nltk
- spacy
- bertopic
- langdetect
- wordsegment

## Usage
1. Place your PDFs in the `papers` folder.
2. Run the script to extract, clean, and analyze the text.
3. The cleaned data will be saved to `csv/all_cleaned_pdf.csv`.
4. Topic modeling results will be visualized using BERTopic.


pip install pdfplumber bertopic nltk pandas spacy numpy matplotlib wordcloud gensim scikit-learn scipy tqdm wordsegment langdetect
python -m spacy download fr_core_web_sm
python -m spacy download nl_core_web_md
python -m spacy download en_core_web_md
Use the following Python version 3.12.0 otherwise the code will not work due to spacy.
"""

'\nNeeded dependancies:\n    - spacy\n    - pdfplumber\n    - bertopic\n    - pandas\n    - numpy\n    - matplotlib\n    - wordcloud\n    - nltk\n    - gensim\n    - scikit-learn\n    - scipy\n    - tqdm\npip install pdfplumber bertopic nltk pandas spacy numpy matplotlib wordcloud gensim scikit-learn scipy tqdm wordsegment\npython -m spacy download fr_core_web_sm\npython -m spaCy download nl_core_web_md\nUse the following Python version 3.12.0 otherwise the code will not work due to spacy.\n'

In [ ]:
import pandas as pd
import os
import re
from collections import Counter
import nltk
from wordsegment import load, segment
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm  # For progress tracking
from langdetect import detect  # For language detection
import spacy  # For Dutch and French tokenization
import bertopic # For topic modeling
from typing import List, Dict, Any
import logging
import fitz  # PyMuPDF
import csv
import unicodedata

c:\Users\Lovepreet Singh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pdfplumber
import re
import pandas as pd
import os
import spacy
from langdetect import detect, LangDetectException

# Load large spaCy models for English, Dutch, and French
nlp_en = spacy.load("en_core_web_lg")  # Large English model
nlp_nl = spacy.load("nl_core_news_lg")  # Large Dutch model
nlp_fr = spacy.load("fr_core_news_lg")  # Large French model

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using pdfplumber."""
    text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

def detect_language(text):
    """
    Detect the language of the text using langdetect.
    
    Args:
        text (str): The input text.
    
    Returns:
        str: The detected language code ('en', 'nl', 'fr') or 'en' as default.
    """
    try:
        language = detect(text)
        # Map langdetect's language codes to spaCy's model names
        if language in ['en', 'nl', 'fr']:
            return language
        else:
            return 'en'  # Default to English if the detected language is not supported
    except LangDetectException:
        return 'en'  # Default to English if language detection fails

def clean_text_with_spacy(text, language):
    """Clean the text using spaCy based on the detected language."""
    if language == 'en':
        nlp = nlp_en
    elif language == 'nl':
        nlp = nlp_nl
    elif language == 'fr':
        nlp = nlp_fr
    else:
        nlp = nlp_en  # Default to English

    doc = nlp(text)
    cleaned_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

def process_pdfs(pdf_paths):
    """Process multiple PDFs and return cleaned text."""
    cleaned_texts = []
    for pdf_path in pdf_paths:
        text = extract_text_from_pdf(pdf_path)
        language = detect_language(text)
        cleaned_text = clean_text_with_spacy(text, language)
        cleaned_texts.append(cleaned_text)
    return cleaned_texts

def write_to_csv(data, output_csv):
    """Write the cleaned text data to a CSV file."""
    df = pd.DataFrame(data, columns=['Cleaned_Text'])
    df.to_csv(output_csv, index=False, encoding='utf-8')

def main(pdf_directory, output_csv):
    """Main function to process PDFs and write to CSV."""
    pdf_paths = [os.path.join(pdf_directory, f) for f in os.listdir(pdf_directory) if f.endswith('.pdf')]
    cleaned_texts = process_pdfs(pdf_paths)
    write_to_csv(cleaned_texts, output_csv)

if __name__ == "__main__":
    pdf_directory = r'papers'  # Replace with the path to your PDF folder
    output_csv = r'csv/cleaned.csv'  # Replace with the desired output CSV file name
    main(pdf_directory, output_csv)

2025-02-24 01:58:47,364 - ERROR - The folder \papers does not exist.
2025-02-24 01:58:47,364 - ERROR - CSV file csv\all_cleaned_pdf.csv does not exist.


# 2.  Load Your Data

Load the articles from your CSV file using pandas. 

In [ ]:
# Load the data
df= pd.read_csv(r'csv/cleaned.csv')
df.head(100)

FileNotFoundError: [Errno 2] No such file or directory: 'csv/all_cleaned_pdf.csv'

### Removing any personal information to anonymize data  

In [ ]:
# Load spaCy model for Named Entity Recognition (NER).
# But because we are using the large English model, it is already loaded in the previous cell.
# We will check again the language of the text and use the appropriate model for NER.

def remove_sensitive_info(text):
    """
    Remove personal information such as names, dates, and numbers from the text.
    Supports English, Dutch, and French languages.
    
    Args:
        text (str): The input text containing potential personal information.
    
    Returns:
        str: The cleaned text with personal information removed.
    """
    if not isinstance(text, str):
        return ""  # Handle non-string values

    # Detect the language of the text
    language = detect_language(text)

    # Load the appropriate spaCy model based on the detected language
    if language == 'en':
        nlp = nlp_en
    elif language == 'nl':
        nlp = nlp_nl
    elif language == 'fr':
        nlp = nlp_fr
    else:
        nlp = nlp_en  # Default to English if the language is not supported

    # Process the text with spaCy
    doc = nlp(text)

    # Remove PERSON names using spaCy NER
    words = [token.text for token in doc if token.ent_type_ != "PER"]

    # Join the remaining words into a single string
    cleaned_text = " ".join(words)

    # Remove dates and numbers using regex
    cleaned_text = re.sub(r'\b\d{1,4}[-/]\d{1,2}[-/]\d{1,4}\b', '', cleaned_text)  # Remove dates (YYYY-MM-DD, DD/MM/YYYY)
    cleaned_text = re.sub(r'\b\d+\b', '', cleaned_text)  # Remove standalone numbers

    return cleaned_text.strip()


# 3. Prepare Your Text Data
We clean up the text
- Remove the name of city, country, geography for better outcome
- Remove special characters (only letters)
- Convert to lower case
- Remove stop words
- Remove words of only one or 2 letters ('a', 'I', at,...)
- Remove very short sentences
- Remove urls 
- use stemming
- do duplicate sentences

In [ ]:
# Load spaCy's English NER model
nlp = spacy.load(detect_language(df['sentence_clean'].iloc[0]) + "_core_web_sm")

# Function to remove geographic entities (cities, countries, locations)
def remove_geographical_entities(text):
    if not isinstance(text, str):
        return ""  # Handle missing or non-string values
    
    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if token.ent_type_ not in ["GPE", "LOC", "FAC"]]
    
    return " ".join(filtered_tokens)

# Apply function to remove cities, countries, and geography
df['sentence_clean'] = df['sentence'].apply(remove_geographical_entities)

# Display a few cleaned sentences
df.head()

In [ ]:
# Load the spaCy model (use 'en_core_web_sm' for small model)
nlp = spacy.load(detect_language(df['sentence_clean'].iloc[0]) + "_core_web_sm")

# Minimum word length threshold
MIN_WORD_SIZE = 4

# Preprocessing function to clean sentences
def preprocess_text(text):
    if not isinstance(text, str):
        return ""  # Handle missing or non-string values
    
    # Remove URLs (http, https, www, etc.)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Normalize Unicode characters and remove diacritics
    text = unicodedata.normalize('NFKD', text)
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    
    # Replace non-alphabetic characters with a space
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # Process the text with spaCy
    doc = nlp(text.lower())

    # Extract tokens, remove stopwords, and apply lemmatization
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop  # Remove stopwords
        and not token.is_punct  # Remove punctuation
        and not token.is_space  # Remove spaces
        and len(token.lemma_) >= MIN_WORD_SIZE  # Remove short words
    ]

    # Remove duplicate words within each sentence (preserve order)
    unique_tokens = list(dict.fromkeys(tokens))

    # Ensure proper spacing between words
    return " ".join(unique_tokens)

# Apply preprocessing function to clean sentences
df['sentence_clean'] = df['sentence_clean'].apply(preprocess_text)

# Remove duplicate sentences (keep one instance of each)
df = df.drop_duplicates(subset=['sentence_clean'])

In [ ]:
# Display the first element of the cleaned text
df.head()

In [ ]:
df['sentence_clean'].is_unique

In [ ]:
df = df[df['sentence_clean'].str.len() >= 15]

In [ ]:
df.info()

### To see how data cleaning looks 

In [ ]:
# Define the output path
output_path = r"cleaned_data.csv"

# Save the cleaned DataFrame to CSV
df.to_csv(output_path, index=False, encoding='utf-8')

print(f" Cleaned DataFrame saved to: {output_path}")

# 4. Initialize and Fit BERTopic
The good thing with BERTopic is that is does most of the work automatically (Meaning, I do not need to bore you to death with details about how it works behind te scenes.)

We need to do 3 things
1. Initialize BERTopic model
2. 'Fit' the model -> this  means: run the model, as you would run a simple linear regression
3. Look at the topics via 

To get started, let's just use the default settings.